# ERA5 Data Preprocessing
This script is doing the necessary preprocessing of the ERA5 Dataset

### Imports

In [1]:
import xarray as xr
import numpy as np
import cdsapi
import zipfile
import os

### The Dataset
The dataset is publicly available on [Copernicus.eu](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-monthly-means?tab=download), but needs a free account to download. There are two possible ways of accessing it: API or manual download. When the internet connection is unstable the api dowload option tends to fail. In that case, manually download the dataset with the configuration as described in the api-call code below and put the downloaded zip under [data/era5_climate_data/](data/era5_climate_data/) and skip **Download Dataset with API**.

First we define the filename and location where to put or load the data from.

In [2]:
dataset_name = "era5-land-monthly-means-1992-2020"
target_zip_file = f'data/era5_climate_data/{dataset_name}.zip'

#### Download Dataset with API
**Requirement:** Setup the cdsapi with your account as described here: https://cds.climate.copernicus.eu/how-to-api

In [ ]:
dataset = "reanalysis-era5-land-monthly-means"
request = {
    "product_type": ["monthly_averaged_reanalysis"],
    "variable": [
        "2m_dewpoint_temperature",
        "2m_temperature",
        "soil_temperature_level_1",
        "soil_temperature_level_4",
        "snow_cover",
        "snow_density",
        "volumetric_soil_water_layer_1",
        "volumetric_soil_water_layer_4",
        "total_precipitation",
        "soil_type"
    ],
    "year": [
        "1992", "1993", "1994", "1995", "1996", 
        "1997", "1998", "1999", "2000", "2001", 
        "2002", "2003", "2004", "2005", "2006", 
        "2007", "2008", "2009", "2010", "2011", 
        "2012", "2013", "2014", "2015", "2016", 
        "2017", "2018", "2019", "2020"
    ],
    "month": [
        "01", "02", "03", "04", "05", "06",
        "07", "08", "09", "10", "11", "12"
    ],
    "time": ["00:00"],
    "data_format": "netcdf",
    "download_format": "zip"
}

client = cdsapi.Client()
result = client.retrieve(dataset, request).download(target_zip_file)

### Load Dataset
1. Unzip the dataset and extract the relevant file.

In [ ]:
with zipfile.ZipFile(target_zip_file, "r") as zip_ref:
    file_list = zip_ref.namelist()
    data_file = next(f for f in file_list if "data" in f)
    zip_ref.extract(data_file, "data/era5_climate_data/")
    os.rename(f"data/era5_climate_data/{data_file}", f"data/era5_climate_data/{dataset_name}.nc")

2. Open the era5 data as an *xarray.Dataset*

In [3]:
ds_era5_global = xr.open_dataset(f"data/era5_climate_data/{dataset_name}.nc")

### Slicing the Dataset
To make the analysis feasible I will focus on a "slice" of the earth that includes Europe and parts of North Africa. 

First we have to convert the longitude dimension of the dataset to run from -180 to 180 instead of 0 to 360.

In [4]:
def to_lon180(ds, lon_name='longitude'):
    lon = ds[lon_name]
    lon180 = ((lon + 180) % 360) - 180
    return ds.assign_coords({lon_name: lon180}).sortby(lon_name)

ds_era5_global = to_lon180(ds_era5_global)

Now we slice from 11°W to 40°E and from 30°N to 72°N. Which converts to longitude=[-11, 40] and latitude=[72, 30]

*Note: This step reduces the dataset from roughly 81GB to 3GB!*

In [5]:
ds_era5_sliced = ds_era5_global.sel(longitude=slice(-11, 40), latitude=slice(72, 30))

### Dataset Information
**Variables:**

1. "t2m" --> 2 metre temperature in Kelvin
2. "d2m" --> 2 metre dewpoint temperature in Kelvin
3. "stl1" --> Soil temperature level 1 in Kelvin
4. "stl4" --> Soil temperature level 4 in Kelvin
5. "snowc" --> Snow cover in %
6. "rsn" --> Snow density in kg/m³
7. "swvl1" --> Volumetric soil water layer 1 in m³/m³
8. "swvl4" --> Volumetric soil water layer 4 in m³/m³
9. "tp" --> Total precipation in m


In [6]:
print(ds_era5_sliced)

<xarray.Dataset> Size: 3GB
Dimensions:     (valid_time: 348, latitude: 420, longitude: 510)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 3kB 1992-01-01 ... 2020-12-01
  * latitude    (latitude) float64 3kB 71.9 71.8 71.7 71.6 ... 30.2 30.1 30.0
    expver      (valid_time) <U4 6kB ...
  * longitude   (longitude) float64 4kB -11.0 -10.9 -10.8 ... 39.7 39.8 39.9
Data variables:
    d2m         (valid_time, latitude, longitude) float32 298MB ...
    t2m         (valid_time, latitude, longitude) float32 298MB ...
    stl1        (valid_time, latitude, longitude) float32 298MB ...
    stl4        (valid_time, latitude, longitude) float32 298MB ...
    snowc       (valid_time, latitude, longitude) float32 298MB ...
    rsn         (valid_time, latitude, longitude) float32 298MB ...
    swvl1       (valid_time, latitude, longitude) float32 298MB ...
    swvl4       (valid_time, latitude, longitude) float32 298MB ...
    tp          (valid_time, latitud

### Resampling to yearly data
Since the PFT dataset only has yearly data, the climate data has to be resampled from monthly means to a yearly timescale. To not loose too much information by only having the yearly mean temperature, seasonal means are computed as well. 

The variable dewpoint temperature is not that useful on its own, so the Vapor Pressure Deficit (VPD) is calculated from it and included in the dataset.

When aggregating the monthly data to yearly data the average has to be weighted by the amount of days in the month to get an exact yearly mean.

##### Helper Functions

In [ ]:
def time_weights_days_in_month(ds):
    """
    Returns a DataArray of weights -> number of days in each month.
    """
    w = xr.DataArray(ds['time'].dt.days_in_month, coords={'time': ds['time']}, dims='time')
    w.name = 'days_in_month'
    return w

def resample_time_weighted_mean(ds, freq, weights):
    """
    Days-weighted mean over resample bins.

    ds: Dataset or DataArray with time dimension
    weights: 1D DataArray over 'time'
    """
    def _group_mean(x):
        w = weights.sel(time=x.time)
        return x.weighted(w).mean('time')

    return ds.resample(time=freq).map(_group_mean)

def calculate_vpd(temp_kelvin, temp_dewpoint_kelvin):
    """
    Vapor pressure deficit (VPD) from air temperature (K) and dewpoint (K).
    Uses Tetens formula.

    es(T)  = 6.112 * exp(17.67 * Tc / (Tc + 243.5))     [hPa]
    ea(Td) = 6.112 * exp(17.67 * Tdc / (Tdc + 243.5))   [hPa]
    VPD    = (es - ea)                                  [hPa]
    """
    # Convert K to °C
    temp_degrees = temp_kelvin - 273.15
    temp_dewpoint_degrees = temp_dewpoint_kelvin - 273.15

    # Tetens formula
    es_hPa = 6.1078 * np.exp(17.269 * temp_degrees / (temp_degrees + 237.3))
    ea_hPa = 6.1078 * np.exp(17.269 * temp_dewpoint_degrees / (temp_dewpoint_degrees + 237.3))

    # Vapor Pressure Deficit es - ea in hPa
    vpd_hPa = (es_hPa - ea_hPa)
    return vpd_hPa

def add_season_and_year_coords(ds):
    """
    Tag each seasonal timestamp with:
      - season: DJF/MAM/JJA/SON
      - season_year: DJF assigned to the year of Jan/Feb (Dec gets +1)
    """
    month = ds['time'].dt.month
    season = xr.full_like(month, '', dtype=object)
    season = xr.where(month==12, 'DJF', season)
    season = xr.where(month== 3, 'MAM', season)
    season = xr.where(month== 6, 'JJA', season)
    season = xr.where(month== 9, 'SON', season)
    season_year = ds['time'].dt.year.where(month != 12, ds['time'].dt.year + 1)
    return ds.assign_coords(season=('time', season.data),
                            season_year=('time', season_year.data))

def convert_to_yearly_time(ds, limit_years=None):
    """
    Convert a seasonal 'long' dataset (1 timestamp per season) into a 'wide' dataset with
    variables named <var>_season_<DJF/MAM/JJA/SON>, indexed by 'year'.
    """
    ds = add_season_and_year_coords(ds)

    # Pick the year index to use
    years = np.unique(ds['season_year'].values)
    if limit_years is not None:
        y0, y1 = limit_years
        years = years[(years >= y0) & (years <= y1)]

    # Define the seasonal prefixes
    seasons = ['DJF','MAM','JJA','SON']
    out = {}

    for v in ds.data_vars:
        for s in seasons:
            sel = ds[v].where(ds['season'] == s, drop=True)

            # Move season-year into the yearly index 
            sel = sel.assign_coords(
                year=('time', ds['season_year'].where(ds['season']==s, drop=True).data)
            ).swap_dims({'time':'year'}).drop_vars('time')

            # Drop the conflicting coords
            sel = sel.reset_coords(['season','season_year'], drop=True)

            # Align to common year index
            sel = sel.reindex(year=years)

            out[f'{v}_season_{s}'] = sel

    yearly = xr.Dataset(out)

    # Annotate with meta information on naming conventions
    for name in yearly.data_vars:
        yearly[name].attrs['cell_method'] = 'seasonal aggregate (DJF/MAM/JJA/SON); DJF labeled by Jan/Feb year'

    return yearly

##### Calculation
Use the helper functions to first calculate the VPD and add it to the existing dataset. Afterwards define which variables should be averaged when aggregating to yearly data and which variables to sum. Total precipation (tp) should be summed instead of averaged to represent the total precipation in a year instead of the average monthly precipation in a year.

In the end the datasets are merged and the time dimension is now represented by an int for the year to match the PFT dataset.

*Info: This takes a considerable amount of time to run.*

**1. Normal yearly aggregation:**

In [ ]:
# Rename time variable
ds_era5_sliced = ds_era5_sliced.rename({'valid_time': 'time'})

# Calculate Vapor Pressure Deficit
ds_era5_sliced['vpd'] = calculate_vpd(temp_kelvin=ds_era5_sliced['t2m'], temp_dewpoint_kelvin=ds_era5_sliced['d2m'])
ds_era5_sliced['vpd'].attrs.update(units='hPa', long_name='Vapor pressure deficit')

# define which variables to take mean from and which to sum
ds_mean = ds_era5_sliced[['t2m','stl1','stl4','snowc','rsn','swvl1','swvl4','vpd']] 
ds_sum = ds_era5_sliced[['tp']]

# Calculate the time weights
time_weights = time_weights_days_in_month(ds_era5_sliced)

# Calculate yearly aggregations
yearly_means = resample_time_weighted_mean(ds_mean, 'YS', time_weights)
yearly_sums = ds_sum.resample(time='YS').sum(skipna=True, min_count=1)



# Merge the aggregated results
ds_era5_sliced_yearly = xr.merge([yearly_means, yearly_sums])
ds_era5_sliced_yearly = ds_era5_sliced_yearly\
                            .assign_coords(year=ds_era5_sliced_yearly['time'].dt.year)\
                            .swap_dims({'time':'year'}).drop_vars('time')

**2. Seasonal Aggregation**

In [ ]:
# Calculate seasonal aggregations
seasonal_means = resample_time_weighted_mean(ds_mean, 'QS-DEC', time_weights)
seasonal_sums  = ds_sum.resample(time='QS-DEC').sum(skipna=True, min_count=1)

# Merge the aggregated results
ds_era5_sliced_seasonal = xr.merge([seasonal_means, seasonal_sums])

# Restructure time coord from seasonal to yearly with variables with different naming
# Have to reduce the years by 1 because of overlap between years with the season December-January-February into the previous year
ds_era5_sliced_seasonal = convert_to_yearly_time(ds_era5_sliced_seasonal, limit_years=(1993, 2020))

**3. Combine yearly and seasonal variables**

In [ ]:
# Align years by slicing this data as well
ds_era5_sliced_yearly = ds_era5_sliced_yearly.sel(year=slice(1993, 2020))

# Merge Datsets
ds_era5_yearly_merged = xr.merge([ds_era5_sliced_yearly, ds_era5_sliced_seasonal])

<xarray.Dataset> Size: 2GB
Dimensions:    (time: 117, latitude: 420, longitude: 510)
Coordinates:
    number     int64 8B 0
  * latitude   (latitude) float64 3kB 71.9 71.8 71.7 71.6 ... 30.2 30.1 30.0
  * longitude  (longitude) float64 4kB -11.0 -10.9 -10.8 ... 39.7 39.8 39.9
  * time       (time) datetime64[ns] 936B 1991-12-01 1992-03-01 ... 2020-12-01
Data variables:
    stl1       (time, latitude, longitude) float64 200MB nan nan ... 284.9 284.9
    stl4       (time, latitude, longitude) float64 200MB nan nan ... 298.5 298.7
    snowc      (time, latitude, longitude) float64 200MB nan nan nan ... 0.0 0.0
    rsn        (time, latitude, longitude) float64 200MB nan nan ... 100.0 100.0
    swvl1      (time, latitude, longitude) float64 200MB nan nan ... 0.02919
    swvl4      (time, latitude, longitude) float64 200MB nan nan ... 0.00267
    vpd        (time, latitude, longitude) float64 200MB nan nan ... 5.589 5.597
    tp         (time, latitude, longitude) float32 100MB nan nan ... 

**4. Save as file for later use**

In [ ]:
ds_era5_yearly_merged.to_netcdf("data/era5_climate_data/era5-land-yearly-means-sliced.nc")